In [1]:
import cv2 as cv2
import mediapipe as mp
#import matplotlib as mpl
import numpy as np
#import torch
#import math
import sys
import pandas as pd
from glob import glob
#import torchvision
#from torchvision import datasets, models, transforms
#from torch.utils.data import DataLoader
import time
from tqdm import tqdm
import copy
#import torch.utils.data as data_utils
import shutil
import os
import csv
import gc
#import objgraph
from sklearn.decomposition import PCA


In [2]:
ROOT_PATH='C:/Users/jimmy/Documents/ORIGINALES_RAW_90/'
FLIP = False
ROTATE = False
ROT_ANG = 0
FRAMES = 90
LANDMARK_IDX = [156, 70, 63, 105, 66, 107, 55, 193, 246, 161, 160, 159, 158, 157, 173, 33, 7, 163, 144, 145, 153, 154, 155, 133, 383, 300, 293, 334, 296, 336, 285, 417, 466, 388, 387, 386, 385, 384, 398, 263, 249, 390, 373, 374, 380, 381, 382, 362, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 95, 88, 178, 87, 14, 317, 402, 318, 324]

In [3]:
def rotate_image(image, angle):
    height, width = image.shape[:2]
    center = (width/2, height/2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    #cv2.imshow('Original Image', image)
    #cv2.imshow('Rotated Image', rotated_image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return rotated_image

In [4]:
def feature_extractor(path, Video, sign, FILENAME_F, FILENAME_HP):
    file = path
    # Split the path in 
    # head and tail pair
    head_tail = os.path.split(file)
    folder = head_tail[0]    

    
    csv_hands_out_path = os.path.join(folder, Video + FILENAME_HP)
    csv_face_out_path = os.path.join(folder, Video + FILENAME_F)
    csv_face_out_path_PCA = csv_face_out_path.replace("BASE", "PCA")

    #csv_pose_out_path = os.path.join(folder, Video + "P.csv")
    pca = PCA(n_components=1)
    if not os.path.exists(folder):
        os.makedirs(folder)   

    if (not os.path.exists(csv_hands_out_path) or (not os.path.exists(csv_face_out_path)) or (not os.path.exists(csv_face_out_path_PCA))):
        holistic_model = mp.solutions.holistic.Holistic()
        capture = cv2.VideoCapture(path)
        frameNr = 1
        hand_landmarks_left = np.zeros(shape=(21,2))
        hand_landmarks_right = np.zeros(shape=(21,2))  
        face_landmarks_list = np.zeros(shape=(68,2))
        face_landmarks_list_pca = np.zeros(shape=(68))
        pose_landmarks_list = np.zeros(shape=(25,2))
        
        csv_out_file_hands = open(csv_hands_out_path, 'w', newline='')
        csv_out_writer_hands = csv.writer(csv_out_file_hands, delimiter=',', quoting=csv.QUOTE_NONE,escapechar=' ')
        
        csv_out_file_face = open(csv_face_out_path, 'w', newline='')
        csv_out_writer_face = csv.writer(csv_out_file_face, delimiter=',', quoting=csv.QUOTE_NONE,escapechar=' ')
        
        csv_out_file_face_pca = open(csv_face_out_path_PCA, 'w', newline='')
        csv_out_writer_face_pca = csv.writer(csv_out_file_face_pca, delimiter=',', quoting=csv.QUOTE_NONE,escapechar=' ')
        
        while (frameNr< (FRAMES+1)):
            success, frame = capture.read()
            if not success:
                csv_out_writer_hands.writerow([sign] + [Video] + [Video + '_'+ str(frameNr)] + 
                hand_landmarks_left.flatten().astype(str).tolist() + 
                            hand_landmarks_right.flatten().astype(str).tolist() + 
                                          pose_landmarks_list.flatten().astype(str).tolist())                
                csv_out_writer_face.writerow([sign] + [Video] + [Video + '_'+ str(frameNr)] + 
                            face_landmarks_list.flatten().astype(str).tolist())
                
                csv_out_writer_face_pca.writerow([sign] + [Video] + [Video + '_'+ str(frameNr)] + 
                            face_landmarks_list_pca.flatten().astype(str).tolist())
                frameNr = frameNr+1        
            elif success:
                BGR_Image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                del frame
                if FLIP:
                    BGR_Image=cv2.flip(BGR_Image,1)
                if ROTATE:
                    BGR_Image = rotate_image(BGR_Image, ROT_ANG)
                results = holistic_model.process(BGR_Image)
                del BGR_Image
                if results.left_hand_landmarks: 
                    hand_landmarks_left = np.array([[lmk.x, lmk.y] for lmk in results.left_hand_landmarks.landmark], dtype=np.float32)
                else:
                    hand_landmarks_left = np.zeros(shape=(21,2))
                if results.right_hand_landmarks:
                    hand_landmarks_right = np.array([[lmk.x, lmk.y] for lmk in results.right_hand_landmarks.landmark], dtype=np.float32)
                else: 
                    hand_landmarks_right = np.zeros(shape=(21,2)) 
                    
                if results.face_landmarks:
                    landmarks = []
                    landmarks_pca = []
                    for index in LANDMARK_IDX:
                        landmark = results.face_landmarks.landmark[index]    
                        if(np.isnan(landmark.x)):
                            print(path)
                        landmarks.append(landmark.x)
                        landmarks.append(landmark.y)  
                        landmarks_pca.append([landmark.x, landmark.y]) 
                    csv_out_writer_face.writerow([sign] + [Video] + [Video + '_'+ str(frameNr)] + landmarks)                
                    pca.fit(landmarks_pca)
                    landmarks_pca = pca.transform(landmarks_pca)
                    landmarks_pca = landmarks_pca.flatten()
                    csv_out_writer_face_pca.writerow([sign] + [Video] + [Video + '_'+ str(frameNr)] + landmarks_pca.flatten().astype(str).tolist())
                else:
                    csv_out_writer_face.writerow([sign] + [Video] + [Video + '_'+ str(frameNr)] + face_landmarks_list.flatten().astype(str).tolist())
                    csv_out_writer_face_pca.writerow([sign] + [Video] + [Video + '_'+ str(frameNr)] + face_landmarks_list_pca.flatten().astype(str).tolist())

                if results.pose_landmarks:
                    pose_landmarks_list = []
                    pose_landmarks = results.pose_landmarks
                    pose_landmarks_list = np.array([[lmk.x, lmk.y] for i, lmk in enumerate(results.pose_landmarks.landmark) if i < 25], dtype=np.float32)             
                csv_out_writer_hands.writerow([sign] + [Video] + [Video + '_'+ str(frameNr)] +
                                              hand_landmarks_left.flatten().astype(str).tolist() +
                                                        hand_landmarks_right.flatten().astype(str).tolist() + 
                                                                pose_landmarks_list.flatten().astype(str).tolist())                       
                frameNr = frameNr+1
                hand_landmarks_right = np.zeros(shape=(21,2))
                hand_landmarks_left = np.zeros(shape=(21,2))
                face_landmarks_list = np.zeros(shape=(68,2)) 
                face_landmarks_list_pca = np.zeros(shape=(68)) 
                pose_landmarks_list = np.zeros(shape=(25,2))
                
        capture.release()        
        del capture
        holistic_model.close()
        del holistic_model
        csv_out_file_face.close()
        del csv_out_file_face
        csv_out_file_hands.close()
        gc.collect()
        

In [5]:
print(os.listdir(ROOT_PATH))

['Ahorro', 'Banco', 'Cajero Automatico', 'Cedula', 'Cuenta Bancaria', 'Cuotas', 'Deposito', 'Deuda', 'Fiador', 'Finanzas', 'Hipoteca', 'Impuesto', 'Intereses', 'Pagar', 'Persona Fisica', 'Persona Juridica', 'Prestamo', 'SINPE Movil', 'Tarjeta de Credito', 'Transferencia']


In [6]:
vid = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(ROOT_PATH, '*','*.mp4'))}

In [7]:
LESCO_DATA = pd.DataFrame.from_dict(vid, orient = 'index').reset_index()

In [8]:
LESCO_DATA.columns = ['Video','path']
classes = LESCO_DATA.Video.str.split('_').str[1]
LESCO_DATA['sign'] = classes

In [9]:
LESCO_DATA.head()

,Video,path,sign
0,Signer10_Ahorro_1,C:/Users/jimmy/Documents/ORIGINALES_RAW_90\Aho...,Ahorro
1,Signer10_Ahorro_10,C:/Users/jimmy/Documents/ORIGINALES_RAW_90\Aho...,Ahorro
2,Signer10_Ahorro_2,C:/Users/jimmy/Documents/ORIGINALES_RAW_90\Aho...,Ahorro
3,Signer10_Ahorro_3,C:/Users/jimmy/Documents/ORIGINALES_RAW_90\Aho...,Ahorro
4,Signer10_Ahorro_4,C:/Users/jimmy/Documents/ORIGINALES_RAW_90\Aho...,Ahorro


In [10]:
len(LESCO_DATA)

1860

In [11]:
LIST = ["_BASE_","_BASE_5_", "_BASE_5N_", "_BASE_FLIP_", "_BASE_FLIP_5_", "_BASE_FLIP_5N_"]
for filename in LIST:
    if '_FLIP_' in filename:
        FLIP = True
    else:
        FLIP = False
    if '_5_' in filename:
        ROTATE = True
        ROT_ANG = 5
    elif '_5N_' in filename:
        ROTATE = True
        ROT_ANG = -5
    else:
        ROTATE = False
        ROT_ANG = 0 
    print(filename + " FLIP " + str(FLIP) + " ROT " + str(ROTATE) + " ANG " + str(ROT_ANG))
    for ID, row in tqdm(LESCO_DATA.iterrows(), total=len(LESCO_DATA)):
        FILENAME_F = filename + "F.csv"
        FILENAME_HP = filename + "HP.csv"
        feature_extractor(row['path'], row['Video'],  row['sign'], FILENAME_F, FILENAME_HP)
        del row

_BASE_ FLIP False ROT False ANG 0


100%|██████████| 1860/1860 [00:00<00:00, 3534.00it/s]


_BASE_5_ FLIP False ROT True ANG 5


100%|██████████| 1860/1860 [00:00<00:00, 3306.81it/s]


_BASE_5N_ FLIP False ROT True ANG -5


...

100%|██████████| 1860/1860 [00:00<00:00, 3442.88it/s]


_BASE_FLIP_5_ FLIP True ROT True ANG 5


100%|██████████| 1860/1860 [00:00<00:00, 3617.16it/s]


_BASE_FLIP_5N_ FLIP True ROT True ANG -5


100%|██████████| 1860/1860 [00:00<00:00, 3074.52it/s]


NameError: name 'Signer10_Ahorro_1_PCA_5N_F' is not defined